In [1]:
import numpy as np
import pandas as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from tensorflow import keras
# 生成对抗式神经网络1

class Identifier(keras.Model):
    '''
    识别器模型生成 
    '''
    def __init__(self, convolutionCount, convKernelSize, convFilterCount, poolingCount, poolingSize, dropoutSize, dropoutRate):
        super().__init__()
        
        self.layersCount = convolutionCount

        #-- create the convolution layers
        self.convolution = []
        for i in range(convolutionCount):
            self.convolution.append(
                keras.layers.Conv2D(
                    filters = convFilterCount[i],
                    kernel_size = convKernelSize[i]
                )
            )

        #-- create the pooling layers
        self.pooling = []
        for i in range(poolingCount):
            self.pooling.append(
                keras.layers.MaxPool2D(
                    pool_size = poolingSize[i]
                )
            )
        
        #-- use the dropout layers
        self.dropout = []
        for i in range(dropoutSize):
            self.dropout.append(
                keras.layers.Dropout(dropoutRate)
            )

        self.outputDense = keras.layers.Dense(1, activation = 'sigmoid')
    
    def call(self, inputDate):
        output = self.convolution[0](inputDate)
        
        # -- 执行前向转播
        for i in range(1, self.layersCount):
            output = self.convolution[i](output)
            output = self.dropout[i](output)
            output = self.pooling[i](output)
        
        output = self.outputDense(output)

        return output


class Generator(keras.layers):
    def __init__(self, inputShape, denseUnits, imgShape):
        self.model = keras.Sequential([
            keras.layers.Dense(
                units = denseUnits * denseUnits * 1,
                use_bias = False,
                input_shape = inputShape,
                activation = 'tanh'
            ),
            
            keras.layers.Dropout(0.3),

            keras.layers.Dense(
                imgShape * imgShape * 1,
                use_bias = False,
            ),

            keras.layers.Reshape((imgShape, imgShape, 1)),
            
            keras.layers.Conv2DTranspose(
                filters = 128,
                kernel_size = (7, 7),
                use_bias = False,
                padding = 'same',
                strides = (2, 2)
            ),

            keras.layers.Dropout(0.3),

            keras.layers.LeakyReLU(),

            keras.layers.Conv2DTranspose(
                filters = 64,
                kernel_size = (5, 5),
                use_bias = False,
                padding = 'same',
                strides = (2, 2)
            ),

            keras.layers.Dropout(0.3),

            keras.layers.LeakyReLU(),

            keras.layers.Conv2DTranspose(
                filters = 1,
                kernel_size = (3, 3),
                use_bias = False,
                padding = 'same',
                strides = (2, 2)
            ),
        ])

    def call(self, inputs):
        return self.model(inputs, training = False)